In [ ]:
import numpy as np
from isaacgym import gymapi, gymtorch
from isaacgym.torch_utils import *
from tqdm import tqdm

In [ ]:
from franka_panda_env import FrankaPandaEnv
from misc import wrap_angle
from motion_primitive_controller import MotionPrimitiveController
from position_controller import PositionController

In [ ]:
env = FrankaPandaEnv(use_gpu=True, use_gpu_pipeline=False, headless=False)

motion_primitive_controller = MotionPrimitiveController(env.states["eef_pos"])
motion_primitive_controller.update(
    current_positions=env.states["eef_pos"],
    current_velocities=env.states["eef_vel"],
    cube_pos_relative=env.states["cube_pos_relative"],
    force=True,
)

position_controller = PositionController(env)

In [ ]:
frame_num = 0
for _ in tqdm(range(20)):
    # Reset all environments
    env.reset_pose("default")
    env.reset_cube()
    motion_primitive_controller.dones *= False
    motion_primitive_controller.update(
        current_positions=env.states["eef_pos"],
        current_velocities=env.states["eef_vel"],
        cube_pos_relative=env.states["cube_pos_relative"],
        force=True,
        epsilon=0.5,
    )
    for _ in range(500):
        indices_done = motion_primitive_controller.update(
            current_positions=env.states["eef_pos"],
            current_velocities=env.states["eef_vel"],
            cube_pos_relative=env.states["cube_pos_relative"],
            epsilon=0.5,
        )
        position_controller.set_target(
            motion_primitive_controller.current_targets[:, :2]
        )
        position_controller.step()
        env.gym.write_viewer_image_to_file(env.viewer, f"./images/{frame_num}.png")
        env.reset_pose("default", indices_done)
        env.reset_cube(indices_done)
        motion_primitive_controller.dones *= False
        frame_num += 1